In [35]:
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg.eigen.arpack import eigsh
import scipy.sparse as sp
import pandas as pd
import pickle
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats.stats import pearsonr
from tensorflow.python.ops.rnn_cell_impl import _RNNCell as RNNCell
import collections
from tensorflow.contrib import rnn
import h5py
import random
import pickle

from utils import normalize_adj, StandardScaler, masked_mae_tf, masked_mae_tf_by_horizon
from gcn import gcn, gcnn_ddgf

In [36]:
file_Name = "data/NYCBikeHourly272.pickle"
fileObject = open(file_Name,'rb') 
hourly_bike = pickle.load(fileObject) 
hourly_bike = pd.DataFrame(hourly_bike)

In [37]:
node_num = 272 # node number 
feature_in = 24 # number of features at each node, e.g., bike sharing demand from past 24 hours
horizon = 1 # the length to predict, e.g., predict the future one hour bike sharing demand
#lstm_steps = 10 # number of cells in the LSTM part

X_whole = []
Y_whole = []

x_offsets = np.sort(
    np.concatenate((np.arange(-feature_in+1, 1, 1),))
)
# Predict the next one hour
y_offsets = np.sort(np.arange(1, 1+ horizon, 1))

min_t = abs(min(x_offsets))
max_t = abs(hourly_bike.shape[0] - abs(max(y_offsets)))  # Exclusive
for t in range(min_t, max_t):
    x_t = hourly_bike.iloc[t + x_offsets, 0:node_num].values.flatten('F')
    y_t = hourly_bike.iloc[t + y_offsets, 0:node_num].values.flatten('F')
    X_whole.append(x_t)
    Y_whole.append(y_t)

X_whole = np.stack(X_whole, axis=0)
Y_whole = np.stack(Y_whole, axis=0)

X_whole = np.reshape(X_whole, [X_whole.shape[0], node_num, feature_in])

num_train = 20000 # Note here actually we use the first 20000 to train the model. The paper mentioned "22304" need to be corrected.
num_val = 2000
num_test = 2000

X_training = X_whole[:num_train, :]
Y_training = Y_whole[:num_train, :]

# shuffle the training dataset
perm = np.arange(X_training.shape[0])
np.random.shuffle(perm)
X_training = X_training[perm]
Y_training = Y_training[perm]

X_val = X_whole[num_train:num_train+num_val, :]
Y_val = Y_whole[num_train:num_train+num_val, :]

X_test = X_whole[num_train+num_val:num_train+num_val+num_test, :]
Y_test = Y_whole[num_train+num_val:num_train+num_val+num_test, :]

scaler = StandardScaler(mean=X_training.mean(), std=X_training.std())

X_training = scaler.transform(X_training)
Y_training = scaler.transform(Y_training)

X_val = scaler.transform(X_val)
Y_val = scaler.transform(Y_val)

X_test = scaler.transform(X_test)
Y_test = scaler.transform(Y_test)

In [32]:
# Create model
def gcn(x, weights, biases, batch_size, n_input, frequency,flag, n_output_vec):
    # Hidden layer with RELU activation
    
   # output_list = tf.Variable(tf.zeros([n_output_vec,1]),dtype=tf.float32) #'Tensor' object does not support item assignment, cant build Ypre

    #Xtem = tf.reshape(x[i,:], [n_input, frequency])
    #Xtem = tf.transpose(Xtem)
    #Atem = tf.convert_to_tensor(A_whole_final, dtype=np.float32)
    #Atem1 = tf.diag(tf.ones([n_input])) # Atem not palying any roles
    #Ytem = tf.reshape(Y[i,:], [n_input, 1])
    #Atem = tf.diag(tf.ones([n_input]))
    #x = tf.reshape(x, [-1, sn, 1]) # 100, 207, 1
    
    # x (?, 207, 12)
    x = tf.transpose(x, [1, 0]) # 207, ?, 12
    #x = tf.reshape(x, [node_num, -1]) # 207, batch*feature_num
    Atem1 = 0.5*(weights['A1'] + tf.transpose(weights['A1']))#+ Atem 
    Atem1 = normalize_adj(Atem1)
    #th = tf.constant(0.01, dtype=tf.float32)
    #where = tf.subtract(Atem1, th)
    #Atem1 = tf.nn.relu(where)

    Z1 = tf.matmul(Atem1, x) #+ biases['b1'] # 207, batch*feature_num  #+ tf.matmul( tf.matmul(weights['A1'], weights['A1']), Xtem)
    Z1 = tf.nn.relu(Z1) # 207*100, hidden1

    
    Atem2 = 0.5*(weights['A2'] + tf.transpose(weights['A2']))#+ Atem 
    Atem2 = normalize_adj(Atem2)
    
    Z2 = tf.matmul(Atem2, x) #+ biases['b2'] 
    Z2 = tf.nn.relu(Z2) # 207*batchsize, hidden2

    Atem3 = 0.5*(weights['A3'] + tf.transpose(weights['A3']))#+ Atem 
    Atem3 = normalize_adj(Atem3)# + biases['b3'] 
    
    Z3 = tf.matmul(Atem3, x)
    Z3 = tf.nn.relu(Z3)
    
    layer_3 = tf.add(tf.add(Z1, Z2), Z3)#tf.nn.relu(Z3) # 207*batchsize, hidden3
    
    
    layer_3 = tf.reshape(layer_3, [-1, sn]) # batchsize, sn*hidden3
    Z4 = layer_3

    
    return Z4

In [33]:
def gcn_corr_final(frequency, horizon, learning_rate, decay,batch_size, n_hidden_vec1,n_hidden_vec2,n_hidden_vec3,keep, early_stop_th,training_epochs, reg1, reg2):
    # set size
    #sn = 3 # station number

    n_output_vec = node_num*horizon
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    predic_res = []
    Y_true = []

    tf.reset_default_graph()

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)
    
    
    batch_size = int(batch_size)
    n_hidden_vec1 = int(n_hidden_vec1)
    n_hidden_vec2 = int(n_hidden_vec2)
    n_hidden_vec3 = int(n_hidden_vec3)
    early_stop_th = int(early_stop_th)
    training_epochs = int(training_epochs)
    
    early_stop_k=0
    display_step = 1
    best_val = 10000
    traing_error = 0
    test_error = 0
    # Network Parameters

    #n_classes = 2 # MNIST total classes (0-9 digits) # n_classes is for classification only

    tf.reset_default_graph()

    # tf Graph input
    X = tf.placeholder(tf.float32, [None, sn, feature_in]) # X is the input signal
    Y = tf.placeholder(tf.float32, [None, n_output_vec]) # y is the regression output
    #num = tf.placeholder(tf.int32,[1, 1] )

    #Xtem = tf.placeholder(tf.float32, [n_input, frequency]) # for each row of X, A, Y, it can be reshaped to Xtem, Atem, Ytem
    #Atem = tf.placeholder(tf.float32, [n_input, n_input]) # 
    #Ytem = tf.placeholder(tf.float32, [n_input, 1]) #

    #Ypre = tf.placeholder(tf.float32, [None, n_output_vec])

    keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

    # Store layers weight & bias
    weights = {
        'h1': tf.Variable(tf.random_normal([frequency, n_hidden_vec1]), dtype=np.float32),
        'h2': tf.Variable(tf.random_normal([n_hidden_vec1, n_hidden_vec2]), dtype=np.float32),
        'h3': tf.Variable(tf.random_normal([n_hidden_vec2, n_hidden_vec3]), dtype=np.float32),
        #'h4': tf.Variable(tf.random_normal([n_hidden_vec3, n_hidden_vec4])),
        #'outg': tf.Variable(tf.random_normal([sn*n_hidden_vec3, n_hidden_vec4]), dtype=np.float32), 
        'out': tf.Variable(tf.random_normal([num_hidden, Y_whole.shape[1]]), dtype=np.float32), # dont forget to change n_hidden_vec1 when add/delete layers
        #'f1': tf.Variable(tf.random_normal([272*n_hidden_vec3, 100])),
        #'f2': tf.Variable(tf.random_normal([50, 10])),
        #'f3': tf.Variable(tf.random_normal([100, 272])),
        'A1': tf.Variable(tf.random_normal([node_num,node_num]), dtype=np.float32),
        'A2': tf.Variable(tf.random_normal([node_num,node_num]), dtype=np.float32),
        'A3': tf.Variable(tf.random_normal([node_num,node_num]), dtype=np.float32),
        #'A4': tf.Variable(tf.random_normal([n_input,n_input])),
        #'h1_wea': tf.Variable(tf.random_normal([9*frequency2, n_hidden_weather1])),
        #'out_wea': tf.Variable(tf.random_normal([n_hidden_weather1, n_input]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1, horizon]), dtype=np.float32),# n_hidden_vec1])),# bias all the same??? dont forget to test
        'b2': tf.Variable(tf.random_normal([1, horizon]), dtype=np.float32), #n_hidden_vec2])),
        'b3': tf.Variable(tf.random_normal([1, horizon]), dtype=np.float32),#n_hidden_vec3])),
        #'b4': tf.Variable(tf.random_normal([n_input, n_hidden_vec4])),
        #'b1': tf.Variable(tf.random_normal([n_input,n_hidden_vec1])),# bias all the same??? dont forget to test
        #'b2': tf.Variable(tf.random_normal([n_input,n_hidden_vec2])),
        #'b3': tf.Variable(tf.random_normal([n_input,n_hidden_vec3])),
        #'bf1': tf.Variable(tf.random_normal([1, 100])), 
        #'bf2': tf.Variable(tf.random_normal([1, 10])), 
        #'bf3': tf.Variable(tf.random_normal([1, 272])), 
        #'boutg': tf.Variable(tf.random_normal([1, n_hidden_vec4]), dtype=np.float32), 
        'bout': tf.Variable(tf.random_normal([Y_whole.shape[1]]), dtype=np.float32), 
        #'b1_wea': tf.Variable(tf.random_normal([1, n_hidden_weather1])), 
        #'bout_wea': tf.Variable(tf.random_normal([1, n_input])), 
    }
    
    
    with tf.variable_scope('lstm'):
        lstm = tf.contrib.rnn.core_rnn_cell.BasicLSTMCell(num_hidden, forget_bias=1.0, state_is_tuple=True)
        rnn_input_seq = tf.unstack(X, feature_in, 2) #
        
        #print (len(rnn_input_seq))
        #Atem = weights['A1']
        for i in range(feature_in):
            rnn_input_seq[i] = gcn(rnn_input_seq[i], weights, biases, batch_size,node_num, frequency, 1, n_output_vec)
            #print (rnn_input_seq[i].shape)
        outputs, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq, dtype=tf.float32)
        output_reshape = tf.reshape(outputs[-1], [-1, num_hidden])
        #print ('123here!!!!!!!!!!!')
        pred = tf.matmul(output_reshape, weights['out']) + biases['bout']
        #print (pred)
        #pred = tf.reshape(pred, [-1, Y_whole.shape[1]])
        #print ('here!!!!!!!!!!!')
        pred = scaler.inverse_transform(pred)
        Y_true_tr = scaler.inverse_transform(Y)

        cost = tf.reduce_mean(tf.pow(pred - Y_true_tr, 2)) 
        #print (cost)
        
    with tf.variable_scope('lstm', reuse=True):
        rnn_input_seq_val = tf.unstack(X, feature_in, 2)
        #Atem = weights['A1']
        for i in range(feature_in):
            rnn_input_seq_val[i] = gcn(rnn_input_seq_val[i], weights, biases, batch_size,node_num,frequency, 2, n_output_vec)
        outputs_val, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq_val, dtype=tf.float32)
        output_reshape = tf.reshape(outputs_val[-1], [-1, num_hidden])

        pred_val = tf.matmul(output_reshape, weights['out']) + biases['bout']
        pred_val = scaler.inverse_transform(pred_val)
        Y_true_val = scaler.inverse_transform(Y)
        
        cost_val =  tf.reduce_mean(tf.pow(pred_val - Y_true_val, 2)) 
        #print ('234here!!!!!!!!!!!')
    with tf.variable_scope('lstm', reuse=True):
        rnn_input_seq_test = tf.unstack(X, feature_in, 2)
        #Atem = weights['A1']
        for i in range(feature_in):
            rnn_input_seq_test[i] = gcn(rnn_input_seq_test[i], weights, biases, batch_size,node_num,frequency, 3, n_output_vec)
        outputs_test, states = tf.contrib.rnn.static_rnn(lstm, rnn_input_seq_test, dtype=tf.float32)
        output_reshape = tf.reshape(outputs_test[-1], [-1, num_hidden])

        pred_tes = tf.matmul(output_reshape, weights['out']) + biases['bout']
        pred_tes = scaler.inverse_transform(pred_tes)
        Y_true_tes = scaler.inverse_transform(Y)
        
        cost_tes = tf.reduce_mean(tf.pow(pred_tes - Y_true_tes, 2)) 
        
        #print ('345here!!!!!!!!!!!')
    #rmse
    #cost_tes = tf.reduce_mean(tf.pow(pred_tes-Y, 2))
    # cross-entropy for classification
    # cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=Y_train))
    # ratio = tf.abs(tf.reduce_sum(pred)-tf.reduce_sum(Y))/tf.reduce_sum(Y)
    #zero = 0
    #ratio = tf.reduce_mean(tf.divide(tf.where(tf.not_equal(Y, zero), np.abs(pred-Y), tf.zeros(Y.get_shape(), tf.float32)), tf.where(tf.not_equal(Y, zero), Y, tf.ones(Y.get_shape(), tf.float32))))
    optimizer = tf.train.RMSPropOptimizer(learning_rate, decay).minimize(cost)
    #optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Initializing the variables
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

    #total_val_cost = []
    #total_val_ratio = []

    # learning start from 

    #index = daily_bike[(daily_bike['year'] == 2016) & (daily_bike['monthofyear'] == 1) & (daily_bike['dayofmonth'] == 1)].index.tolist()[0]
    #A_hat = normalize_adj(corr_matrix_trips)
    #print(A_hat)
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        sess.run(init)

        for epoch in range(training_epochs):

            avg_cost = 0.
            total_batch = int(num_train/batch_size) #int(num_train/batch_size)

            for i in range(total_batch):
                #print (Y_training[i*batch_size:(i+1)*batch_size,].size())
                #num = batch_size
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[i*batch_size:(i+1)*batch_size,], 
                                                      Y: Y_training[i*batch_size:(i+1)*batch_size,],  
                                                              keep_prob: keep})
                #print (preds)
                #print (trueval)
                #print ("Epoch:", '%04d' % (epoch+1), "batch: ", i, "batch cost=", \
                #    "{:.9f}".format(c))
                #print ('here!!!!!!!!!!!!!!!!')
                avg_cost += c * batch_size #/ total_batch 
                #Display logs per epoch step
                
            # rest part of training dataset
            #num = num_train - total_batch*batch_size 
            if total_batch * batch_size != num_train:
                _, c, preds, trueval = sess.run([optimizer, cost, pred, Y_true_tr], feed_dict={X: X_training[total_batch*batch_size:num_train,], 
                                          Y: Y_training[total_batch*batch_size:num_train,],
                                                  keep_prob: keep})
                avg_cost += c * (num_train - total_batch*batch_size)
            
            avg_cost = np.sqrt(avg_cost / num_train)
            
            if epoch % display_step == 0:
                print ("Epoch:", '%04d' % (epoch+1), "Training RMSE=", \
                    "{:.9f}".format(avg_cost)) #np.sqrt(avg_cost)
                
            # also use batch to save memory
            # validation
            c_val = 0.
            total_bat_val = int(num_val/batch_size)
            for i in range(total_bat_val):
                #num = batch_size
                c_val_b = sess.run([cost_val], feed_dict={X: X_val[i*batch_size:(i+1)*batch_size,], 
                                                          Y: Y_val[i*batch_size:(i+1)*batch_size,],   keep_prob:1})
                c_val += c_val_b[0]*batch_size
            
            if total_bat_val * batch_size != num_val:
                #num = num_val - total_bat_val*batch_size
                c_val_b = sess.run([cost_val], feed_dict={X: X_val[total_bat_val*batch_size:num_val,], 
                                                          Y: Y_val[total_bat_val*batch_size:num_val,],  keep_prob:1})
                c_val += c_val_b[0] * (num_val - total_bat_val*batch_size)
                
            c_val = np.sqrt(c_val / num_val)
            
            print("Validation RMSE: ", c_val)
            
            # test
            c_tes = 0.
            total_bat_test = int(num_test/batch_size)
            
            pre_test_tem = [] # save the prediction results
            Y_tes_true = []
            
            for i in range(total_bat_test):
                #num = batch_size
                c_tes_b, pred_tes1, Y_tes_batch = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test[i*batch_size:(i+1)*batch_size,],
                                                                               Y: Y_test[i*batch_size:(i+1)*batch_size,],  keep_prob: 1})
                c_tes += c_tes_b*batch_size

                #print (cost_h)
                #print (pred_tes1.shape)
                pre_test_tem.append(pred_tes1)
                Y_tes_true.append(Y_tes_batch)
                
            if total_bat_test * batch_size != num_test:
                #num = num_test - total_bat_test*batch_size
                c_tes_b, pred_tes1, Y_tes_batch = sess.run([cost_tes, pred_tes, Y_true_tes], feed_dict={X: X_test[total_bat_test*batch_size:num_test,],
                                                                               Y: Y_test[total_bat_test*batch_size:num_test,],  keep_prob: 1})
                c_tes += c_tes_b * (num_test - total_bat_test*batch_size) 
                 
                
                #print (pred_tes1.shape)
                pre_test_tem.append(pred_tes1)
                Y_tes_true.append(Y_tes_batch)
            
            #print (c_tes_h.shape)
            pre_test_tem = np.concatenate(pre_test_tem, axis = 0)
            #print (pre_test_tem.shape)
            Y_tes_true = np.concatenate(Y_tes_true, axis = 0)
            
            c_tes = np.sqrt(c_tes / num_test)
            #c_tes_h = c_tes_h / num_test
            
            print("Test RMSE: ", c_tes)
            #print("predic step: ", cost_by_hor)

            if c_val < best_val:
                best_val = c_val
                #saver.save(sess, './bikesharing_graph_2_th_point1')
                test_error = c_tes
                
                traing_error = avg_cost#np.sqrt(avg_cost)
                early_stop_k = 0 # reset to 0
                #print (pred_tes1)
                predic_res = pre_test_tem
                Y_true = Y_tes_true
                #predic_step = cost_by_hor

            # early stopping
            if c_val >= best_val:
                early_stop_k += 1

            # threshold
            if early_stop_k == early_stop_th:
              #  print ("early stopping...")
                break
            

        print("epoch is ", epoch)
        print("training RMSE is ", traing_error)
        print("Optimization Finished! the lowest validation RMSE is ", best_val)#(np.sqrt(best_val)))
        print("The test RMSE is ", test_error)#(np.sqrt(test_error)))
    
    #test_Y = Y_test
    #test_error = np.sqrt(test_error)
    return test_error, predic_res, Y_true#, A1#, predic_step#, A2

In [34]:

import datetime
#freq_max = 12
#time_step = 12
learning_rate = 0.01
decay = 0.9 
batch_size = 1000
num_hidden = 50 # number of hiddent units in LSTM Cell
early_stop_th = 150
training_epochs = 200
keep = 1#0.2
#time_step_max = 10
frequency = 24
sn = 272 # station num

n_hidden_vec1 = 10
n_hidden_vec2 = 10
n_hidden_vec3 = 10
#n_hidden_vec4 = 10
#num = 0
#All_pred = np.empty([2000, 207])
#All_Y = np.empty([2000, 207])

#24*90
#step = 0
#gap = 100
#training = 0.7
#validation = 0.1
#test = 0.2

#gcn_corr_eval(7, 0.01, 0.5, 100, 0.4, 10, 5, 5, 0.2, 50, 500)

reg1 = 0
reg2 = 0

rep = 1 # repeating times

#total_sn = 0
#num_iter = 50
#init_points = 200


# stdbscan
#spatial_threshold = 300
#temporal_threshold = 300
#min_neighbors = 1 # number of neighbor

#frequency2 = skip1 + freq_max + training

#while step < 2000:

#hourly_bike_cluster = hourly_bike
best = -10000
pre_best = []
test_Y_best = []
test_error_best = 1000
A1_best = []
# A2_best = []
for i in range(rep):
    a = datetime.datetime.now()
    test_error, predic_res, Y_true = gcn_corr_final(frequency, horizon, learning_rate, decay, batch_size, n_hidden_vec1,
                                                                n_hidden_vec2, n_hidden_vec3, keep, early_stop_th, training_epochs, reg1, reg2)
    #val_error, predic_res, test_Y,test_error=gcn_corr_final(a['frequency'], a['learning_rate'], a['decay'], a['batch_size'], a['n_hidden_vec1'], a['n_hidden_vec2'], a['n_hidden_vec3'], a['keep'], a['early_stop_th'], a['training_epochs'], a['reg'])
    #print ("finished A running: ", i)
    b = datetime.datetime.now()
    print(b-a)
    


    

Epoch: 0001 Training RMSE= 7.860712308
Validation RMSE:  7.442989599431542
Test RMSE:  6.711288780459872
Epoch: 0002 Training RMSE= 6.860021080
Validation RMSE:  7.0747928818219625
Test RMSE:  6.368018123064179
Epoch: 0003 Training RMSE= 6.456604101
Validation RMSE:  6.6913539441275125
Test RMSE:  5.958595145637866
Epoch: 0004 Training RMSE= 6.083514651
Validation RMSE:  6.345785481536982
Test RMSE:  5.612624478393457
Epoch: 0005 Training RMSE= 5.786654315
Validation RMSE:  5.98587447620931
Test RMSE:  5.060063564670252
Epoch: 0006 Training RMSE= 5.559089610
Validation RMSE:  5.743988045459516
Test RMSE:  4.754923427217946
Epoch: 0007 Training RMSE= 5.413899790
Validation RMSE:  5.6455235226169584
Test RMSE:  4.718099385054429
Epoch: 0008 Training RMSE= 5.277343000
Validation RMSE:  5.47359761780612
Test RMSE:  4.408363322472679
Epoch: 0009 Training RMSE= 5.197372611
Validation RMSE:  5.460581493643169
Test RMSE:  4.527545019354608
Epoch: 0010 Training RMSE= 5.143364739
Validation RMSE

Epoch: 0080 Training RMSE= 4.115404076
Validation RMSE:  5.9828266102785586
Test RMSE:  5.311113064303143
Epoch: 0081 Training RMSE= 4.084836933
Validation RMSE:  5.752305273600608
Test RMSE:  5.2695583967749595
Epoch: 0082 Training RMSE= 4.079790621
Validation RMSE:  6.059531123966753
Test RMSE:  5.449536211070763
Epoch: 0083 Training RMSE= 4.060493431
Validation RMSE:  5.933778660738966
Test RMSE:  5.4627622248999135
Epoch: 0084 Training RMSE= 4.054962167
Validation RMSE:  5.982930858255266
Test RMSE:  5.447510285352474
Epoch: 0085 Training RMSE= 4.056015368
Validation RMSE:  5.902810038559462
Test RMSE:  5.488295498758194
Epoch: 0086 Training RMSE= 4.039342201
Validation RMSE:  5.978517539202568
Test RMSE:  5.386768498512572
Epoch: 0087 Training RMSE= 4.026192365
Validation RMSE:  5.851527622663917
Test RMSE:  5.469946245421011
Epoch: 0088 Training RMSE= 4.020872613
Validation RMSE:  6.078527520587735
Test RMSE:  5.462876920986342
Epoch: 0089 Training RMSE= 4.014905487
Validation RM

Epoch: 0159 Training RMSE= 3.667903561
Validation RMSE:  6.289288591384384
Test RMSE:  5.972706706580529
Epoch: 0160 Training RMSE= 3.664361996
Validation RMSE:  6.105793949772632
Test RMSE:  5.942980194782837
Epoch: 0161 Training RMSE= 3.660426508
Validation RMSE:  6.321330022774949
Test RMSE:  5.950368488345467
Epoch: 0162 Training RMSE= 3.654719367
Validation RMSE:  6.230232105125798
Test RMSE:  5.986452065907013
Epoch: 0163 Training RMSE= 3.661800820
Validation RMSE:  6.182173314577931
Test RMSE:  5.8933401958431375
Epoch: 0164 Training RMSE= 3.648265121
Validation RMSE:  6.205883114391713
Test RMSE:  6.017652927764545
Epoch: 0165 Training RMSE= 3.647251180
Validation RMSE:  6.298700186442937
Test RMSE:  5.934285468239179
Epoch: 0166 Training RMSE= 3.657208048
Validation RMSE:  6.193944330178989
Test RMSE:  6.033008062260225
epoch is  165
training RMSE is  5.083405542121361
Optimization Finished! the lowest validation RMSE is  5.308986624017716
The test RMSE is  4.212664418167147
0

In [ ]:

#lstm cell num 10, hidden num [10, 10, 10], batchsize 1000, hidden state 50: 2.40

#lstm cell num 10, hidden num [10, 10, 10], batchsize 1000, hidden state 50, no relu between layers: 2.41

#lstm cell num 10, hidden num [10, 10, 10], batchsize 1000, hidden state 50, sum layers: 2.4+

#lstm cell num 10, hidden num 1, batchsize 1000, hidden state 50, sum layers: 3.04, 2.4
#lstm cell num 10, hidden num 1, batchsize 1000, hidden state 100, sum layers: 
#lstm cell num 10, hidden num 1, batchsize 1000, hidden state 20, sum layers: 
#lstm cell num 10, hidden num 1, batchsize 2000, hidden state 50, sum layers: 

In [ ]:
#np.savetxt("lstm_gcnn_prediction_1.95.csv", predic_res, delimiter = ',')
#np.savetxt("lstm_gcnn_prediction_1.95_Y.csv", Y_true, delimiter = ',')